<a href="https://colab.research.google.com/github/ArunBharathRam/test/blob/master/Finalproject_braintumour_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.listdir('/content/drive/MyDrive'))


['New document.exe', 'Scrap.shs', 'student affidavit.pdf', 'STATISTICS OF INDIAN CINEMA.pptx', 'Resume.gdoc', 'Group 21(review-2).pptx', 'ANNEXURE 2.docx.gdoc', 'aadhar front (1).jpg', 'New Doc 2017-10-10 (1).jpg', 'aadhar back (1).jpg', 'Char verification front.pdf', 'char form back_1.pdf', 'char form back_2.pdf', 'char form back_3.pdf', 'char form back_4.pdf', 'char form back_5.pdf', 'char form back_6.pdf', 'char form back_8.pdf', 'New Doc 2017-10-09.pdf', 'char form back_9.pdf', 'aadhar1.pdf', 'aadhar2.pdf', 'char front.pdf', 'char front_2.pdf', 'characterform2.pdf', 'adhar card1 - Page 1.pdf', 'adhar card1 - adhar card 2.pdf', 'edited', 'page 2 bgc excel.pdf', 'page 3 bgc (1).pdf', 'page 3 bgc.pdf', 'OCT_2017.pdf', 'SpiceJet_E-ticket_PNR K2K4RB - 28 Jan 2018 Madurai-Chennai for MR. KUMAR A S.PDF', 'SpiceJet_Mobile_PNR K2K4RB - 28 Jan 2018 Madurai-Chennai for MR. KUMAR A S.PNG', 'IMG_4618.JPG', 'IMG_20170729_145000.jpg', 'IMG_20170729_144930.jpg', 'IMG_20180204_222358_594.jpg', 'IMG

In [ ]:
dataset_path = "/content/drive/MyDrive/brain_tumor_dataset (3)"
print(os.listdir(dataset_path))

['Testing', 'Training', 'Validation']


In [ ]:
train_dir = "/content/drive/MyDrive/Training"
test_dir = "/content/drive/MyDrive/Testing"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Search entire MyDrive for the folder names
for root, dirs, files in os.walk('/content/drive', topdown=True):
    if "Training" in dirs or "Testing" in dirs:
        print("FOUND HERE:", root)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
FOUND HERE: /content/drive/MyDrive/brain_tumor_dataset (3)
FOUND HERE: /content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset
FOUND HERE: /content/drive/.Encrypted/MyDrive/brain_tumor_dataset (3)
FOUND HERE: /content/drive/.Encrypted/MyDrive/Final_project_datascience/brain_tumor_dataset


In [ ]:
train_dir = "/content/drive/MyDrive/brain_tumor_dataset (3)/Training"
test_dir  = "/content/drive/MyDrive/brain_tumor_dataset (3)/Testing"


In [ ]:
import os

print("Option 1:", os.listdir("/content/drive/MyDrive/brain_tumor_dataset (3)"))
print("Option 2:", os.listdir("/content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset"))


Option 1: ['Testing', 'Training', 'Validation']
Option 2: ['Testing', 'Training']


In [ ]:
train_dir = "/content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset/Training"
val_dir   = "/content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset/Validation"
test_dir  = "/content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset/Testing"


In [ ]:
train_dir = "/content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset/Training"
test_dir  = "/content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset/Testing"


In [ ]:
#import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


In [ ]:
device = torch.device("cpu")
print("Using CPU only")


Using CPU only


In [ ]:
 #Data Preprocessing & Augmentation
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 5
LR = 0.0003



In [ ]:
#transforms

from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),           # Standard size for ResNet
    transforms.RandomHorizontalFlip(),       # Augmentation
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])



In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader, WeightedRandomSampler

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset  = datasets.ImageFolder(root=test_dir, transform=test_transform)

# Compute class weights for imbalance
class_counts = [0]*len(train_dataset.classes)
for _, label in train_dataset.samples:
    class_counts[label] += 1

class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
weights = [class_weights[label] for _, label in train_dataset.samples]
sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print("Training images:", len(train_dataset))
print("Testing images:", len(test_dataset))
print("Classes:", train_dataset.classes)


Training images: 5693
Testing images: 1311
Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [ ]:
import torch.nn as nn
from torchvision import models

# Load pretrained ResNet18
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))  # 4 classes

model = model.to(device)

print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
from torchvision import transforms
train_transform = transforms.Compose([
    transforms.Resize((96,96)),      # Smaller = faster
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((96,96)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])


In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_dir = "/content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset/Training"
test_dir = "/content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset/Testing"

train_dataset = ImageFolder(root=train_dir, transform=train_transform)
test_dataset = ImageFolder(root=test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet18(weights=None)  # Start fresh to avoid deprecated weights issues
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4 classes
model = model.to(device)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)


In [ ]:
from sklearn.metrics import classification_report

def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(outputs.argmax(1) == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        # Validation
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                preds = outputs.argmax(1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}")
        print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))


In [ ]:
r# ===============================
# Step 0: Imports & Setup
# ===============================
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import os

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ===============================
# Step 1: Paths
# ===============================
train_dir = "/content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset/Training"
test_dir  = "/content/drive/MyDrive/Final_project_datascience/brain_tumor_dataset/Testing"

# ===============================
# Step 2: Transforms
# ===============================
train_transform = transforms.Compose([
    transforms.Resize((96, 96)),  # smaller = faster
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ===============================
# Step 3: Datasets & Loaders
# ===============================
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
test_dataset  = datasets.ImageFolder(test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

print(f"Training images: {len(train_dataset)}")
print(f"Testing images: {len(test_dataset)}")
print("Classes:", train_dataset.classes)

# ===============================
# Step 4: Model (ResNet18)
# ===============================
model = models.resnet18(weights=None)  # start from scratch
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)      # 4 classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ===============================
# Step 5: Training Loop
# ===============================
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}] | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.4f}")

# ===============================
# Step 6: Evaluation on Test Set
# ===============================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

# ===============================
# Step 7: Save the Model
# ===============================
torch.save(model.state_dict(), "brain_tumor_resnet18.pth")
print("Model saved as brain_tumor_resnet18.pth")


Using device: cpu
Training images: 5693
Testing images: 1311
Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch [1/5] | Loss: 0.5182 | Accuracy: 0.7948
Epoch [2/5] | Loss: 0.2729 | Accuracy: 0.8978
Epoch [3/5] | Loss: 0.1896 | Accuracy: 0.9359
Epoch [4/5] | Loss: 0.1276 | Accuracy: 0.9547
Epoch [5/5] | Loss: 0.1021 | Accuracy: 0.9665

Classification Report:
              precision    recall  f1-score   support

      glioma       0.97      0.89      0.93       300
  meningioma       0.75      0.96      0.84       306
     notumor       0.99      0.84      0.91       405
   pituitary       0.99      0.99      0.99       300

    accuracy                           0.91      1311
   macro avg       0.92      0.92      0.92      1311
weighted avg       0.93      0.91      0.92      1311

Model saved as brain_tumor_resnet18.pth
